In [7]:
%run config.py
%run dataset.py
%run models.py

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from models import GNN
from dataset import MyGraphDataset, load_wave_data, normalise_data
from config import Config
import torch
import math
import pickle
from scipy.io import savemat

cfg = Config(
    snapshots=3,
    data_dir="sample_data",
    data_fname="sample_wave_data.nc", 
    neighbourhood_size=5,
    normalization="norm_01",
    train_period=(0, 90),
    test_period=(90, 119),
    convolution_kernels = (64, 128),
    node_var_observ= ['hs', 'ub_bot', 'wlen', 'pwave_bot', 'tpeak', 'dirm'],
    node_var_target=['hs', 'ub_bot', 'wlen', 'pwave_bot', 'tpeak', 'dirm'],
    train_batch_size=4,
    forward_time=1,
    max_epoches=10,
    test_batch_size=3,
)

In [3]:
d = load_wave_data(cfg)
d['tpeak'].shape

(120, 300)

In [11]:
def save_data(data, filename):
    with open(filename, 'wb') as file:
        pickle.dump(data, file)   
    
def main(config:Config):
    trn_dl, test_dl = prepare_train_test_dataloaders(cfg)
    # setup the model and optimiser
    model = GNN(cfg)
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate)
    loss_fn = torch.nn.MSELoss()
    
    r2_scores = []
    mse =[]
 
    # Create lists to store training and testing data
    #train_data = []
    test_data = []
    
    for epoch in range(cfg.max_epoches):
        for i, (data_x, y) in enumerate(trn_dl):
            optimizer.zero_grad()            #clar gradients from previous iteration

            # compute the loss for this batch
            # TODO: to wrap in a proper loss function
            tmp = model(data_x.x, data_x.edge_index)
            batch_pred = unbatch(tmp, data_x.batch)
            
            loss = 0
            for pi, yi in zip(batch_pred, y):
                loss += loss_fn(pi, yi)

            
            loss.backward()                   #gradients of the model's parameters are computed with respect to the loss using backpropagation
            optimizer.step()                  #The optimizer updates the model's parameters based on the computed gradients using the chosen optimization algorithm (e.g., Adam)
            
            # Calculate R-squared
            y_true_int = y.detach().numpy()
            y_true = y_true_int.reshape(-1, 6)
            y_pred = torch.cat(batch_pred).detach().numpy()
            ss_res = np.sum((y_true - y_pred)**2)
            ss_tot = np.sum((y_true - np.mean(y_true))**2)
            r2 = 1 - (ss_res / ss_tot)
            r2_scores.append(r2)
            
    
            print(f"epoch {epoch}, batch {i}: loss {loss.item():.5f}, R2 {r2:.5f}")
    
    # Test the model using test data
    model.eval()  # Set model to evaluation mode
    test_r2 = 0
    test_loss = 0
    for i, (data_x, y) in enumerate(test_dl):
        test_data_out = model(data_x.x, data_x.edge_index)
        batch_pred = unbatch(test_data_out, data_x.batch)
        
        y_true_int = y.detach().numpy()
        y_true = y_true_int.reshape(-1, 6)
        y_pred = torch.cat(batch_pred).detach().numpy()
        ss_res = np.sum((y_true - y_pred)**2)
        ss_tot = np.sum((y_true - np.mean(y_true))**2)
        r2 = 1 - (ss_res / ss_tot)
        test_r2 += r2
        mse = np.mean((y_true - y_pred)**2)
        test_loss += mse
        
        # Create dictionaries for lon, lat, time, and variable data
        # Save for prediction results
        data = load_wave_data(cfg)
        lon = data['lon']
        lat = data['lat']
        time= data['t']
        lon_lat_time_data = {
            'lon': lon,  # Use data_x.node_index as an index to extract lon values
            'lat': lat,  # Use data_x.node_index as an index to extract lat values
            'time': time,  # Use data_x.time as an index to extract time values
        }
        
        variable_pred_data = {
            'hs': y_pred[:, 0],  # Adjust the index as needed
            'ub_bot': y_pred[:, 1],  # Adjust the index as needed
            'wlen': y_pred[:, 2],  # Adjust the index as needed
            'pwave_bot': y_pred[:, 3],  # Adjust the index as needed
            'tpeak': y_pred[:, 4],  # Adjust the index as needed
            'dirm' : y_pred[:, 5],
        }
        
        variable_true_data = {
            'hs': y_true[:, 0],  # Adjust the index as needed
            'ub_bot': y_true[:, 1],  # Adjust the index as needed
            'wlen': y_true[:, 2],  # Adjust the index as needed
            'pwave_bot': y_true[:, 3],  # Adjust the index as needed
            'tpeak': y_true[:, 4],  # Adjust the index as needed
            'dirm' : y_true[:, 5],
        }
        
        test_data.append({
            'lon_lat_time_data': lon_lat_time_data,
            'variable_pred_data': variable_pred_data,
            'variable_true_data': variable_true_data
        })
                
    test_r2 /= len(test_dl)    
    # Calculate average test loss (mean squared error)
    average_mse = test_loss / len(test_dl)
    # Calculate RMSE from average MSE
    rmse = math.sqrt(average_mse)

    print(f"Test R2: {test_r2:.5f}, loss {rmse:.5f}")     
    
    # Convert the lists to tensors if needed
    test_data = np.array(test_data)
    
    # Save training and testing data to separate files
    save_data(test_data, 'test.pkl')

    print("testing data saved.")
  


Verify the algorithm on a small dataset of 120 time steps, 90 for training and 30 
for test.

On this toy data, the R2 on the test split is ~0.75.


In [ ]:
main(cfg)